In [ ]:
# 1. Setup and Configuration
import sys
import sqlite3
import json
import re
from pathlib import Path
from collections import Counter, defaultdict
from typing import List, Tuple, Dict, Any
from datetime import datetime

print("🏷️ Task 2: CoNLL Format Labeling for Amharic NER")
print("=" * 60)
print("Setting up CoNLL labeling system...")

# Add project root to path
project_root = Path.cwd().parent if 'notebook' in str(Path.cwd()) else Path.cwd()
sys.path.insert(0, str(project_root))

print(f"✅ Project root: {project_root}")
print("Ready to start CoNLL labeling!")


In [ ]:
# 2. Load and Preview Data
print("📊 Loading data for CoNLL labeling...")

# Try to load from existing databases
data_sources = [
    project_root / "data/processed/amharic_ecommerce.db",
    project_root / "data/historical_messages.db", 
    project_root / "data/demo.db"
]

messages = []
for db_path in data_sources:
    if db_path.exists():
        print(f"📂 Found database: {db_path}")
        try:
            conn = sqlite3.connect(db_path)
            cursor = conn.cursor()
            
            # Check available tables
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
            tables = [row[0] for row in cursor.fetchall()]
            print(f"   Available tables: {tables}")
            
            # Try to load messages from any available table
            for table in ['processed_messages', 'historical_messages', 'messages']:
                if table in tables:
                    cursor.execute(f"SELECT * FROM {table} LIMIT 50")
                    rows = cursor.fetchall()
                    if rows:
                        cursor.execute(f"PRAGMA table_info({table})")
                        columns = [row[1] for row in cursor.fetchall()]
                        messages = [dict(zip(columns, row)) for row in rows]
                        print(f"   ✅ Loaded {len(messages)} messages from {table}")
                        break
            
            conn.close()
            if messages:
                break
                
        except Exception as e:
            print(f"   ❌ Error loading from {db_path}: {e}")

print(f"\n📋 Data Summary:")
print(f"   Messages loaded: {len(messages)}")
if messages:
    sample_msg = messages[0]
    print(f"   Sample fields: {list(sample_msg.keys())}")
    text_field = None
    for field in ['text', 'original_text', 'message']:
        if field in sample_msg:
            text_field = field
            break
    
    if text_field:
        print(f"   Text field: '{text_field}'")
        print(f"   Sample text: {sample_msg[text_field][:100]}...")
    else:
        print("   ⚠️ No text field found")


In [ ]:
# 3. Run CoNLL Labeling System
print("🏷️ Running CoNLL labeling process...")

# Run the labeling script
import subprocess
result = subprocess.run(['python3', 'task2_conll_labeling.py'], 
                       capture_output=True, text=True, cwd=project_root)

print("STDOUT:")
print(result.stdout)

if result.stderr:
    print("STDERR:")
    print(result.stderr)

print(f"Return code: {result.returncode}")

# Check if files were created
output_dir = project_root / "data/conll_labeled"
conll_file = output_dir / "amharic_ecommerce_conll.txt"
stats_file = output_dir / "labeling_statistics.json"

print(f"\n📁 Output Files:")
print(f"   CoNLL file exists: {conll_file.exists()}")
print(f"   Stats file exists: {stats_file.exists()}")

if stats_file.exists():
    with open(stats_file, 'r', encoding='utf-8') as f:
        stats = json.load(f)
    print(f"\n📊 Labeling Statistics:")
    print(f"   Messages processed: {stats['total_messages_processed']}")
    print(f"   Total tokens: {stats['total_tokens']:,}")
    print(f"   Total entities: {stats['total_entities']}")
    print(f"   Entity distribution: {stats['entity_counts']}")


In [ ]:
# 4. Validate CoNLL Format
print("🧪 Validating CoNLL format...")

# Run the validation script
result = subprocess.run(['python3', 'test_conll_format.py'], 
                       capture_output=True, text=True, cwd=project_root)

print("STDOUT:")
print(result.stdout)

if result.stderr:
    print("STDERR:")
    print(result.stderr)

print(f"Return code: {result.returncode}")

# Load validation report
validation_file = output_dir / "validation_report.json"
if validation_file.exists():
    with open(validation_file, 'r', encoding='utf-8') as f:
        validation_report = json.load(f)
    
    print(f"\n✅ Validation Summary:")
    checks = validation_report['requirements_check']
    print(f"   Message count (30-50): {'✅' if checks['message_count_ok'] else '❌'}")
    print(f"   CoNLL format valid: {'✅' if checks['format_valid'] else '❌'}")
    print(f"   All entity types found: {'✅' if checks['all_entity_types_found'] else '❌'}")
    
    if all(checks.values()):
        print("\n🎉 Task 2 PASSED: Ready for NER training!")
    else:
        print("\n⚠️ Task 2 ISSUES: Some requirements need attention")


In [ ]:
# 5. Preview CoNLL Output
print("👀 Previewing CoNLL format output...")

if conll_file.exists():
    with open(conll_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    print(f"📄 File: {conll_file}")
    print(f"📊 Total lines: {len(lines)}")
    
    print(f"\n📝 First 50 lines:")
    print("=" * 50)
    for i, line in enumerate(lines[:50]):
        print(f"{i+1:2d}: {line.rstrip()}")
    
    print(f"\n..." + "." * 40)
    print(f"📄 Full file contains {len(lines)} lines")
    
    # Count entity examples
    entity_lines = [line for line in lines if '\t' in line and not line.startswith('#')]
    total_tokens = len(entity_lines)
    entity_tokens = len([line for line in entity_lines if not line.strip().endswith('\tO')])
    
    print(f"\n📊 Token Statistics:")
    print(f"   Total tokens: {total_tokens}")
    print(f"   Entity tokens: {entity_tokens}")
    print(f"   Entity coverage: {(entity_tokens/total_tokens)*100:.1f}%")
else:
    print(f"❌ CoNLL file not found: {conll_file}")
